In [1]:
# !pip install pygame

try:
    import pygame
    import pygame.freetype
    from pygame.locals import *
    import os
    import sys
    import random
    from enum import Enum
    from collections import Counter
except ImportError as err:
    print("couldn't load module. %s" % (err))
    sys.exit(2)

pygame.freetype.init()
FONT = pygame.freetype.Font(None, size=32)
FPS = 30
SPACE = 10
SCREEN_WIDTH = 1000
SCREEN_HEIGHT = 600
CARD_WIDTH = 70
CARD_HEIGHT = 105
BOARD_WIDTH = (CARD_WIDTH + SPACE) * 5 - SPACE
BOARD_HEIGHT = SCREEN_HEIGHT
BLACK = (  0,   0,   0)
WHITE = (255, 255, 255)
GREEN = (  0,  60,   0)
LIME  = ( 50, 205,  50)
RED   = (255,   0,   0)


pygame 1.9.6
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
BUTTON_HEIGHT = 40
BUTTON_WIDTH = 120
BLACK     = (  0,   0,   0)
WHITE     = (255, 255, 255)
DARKGRAY  = ( 64,  64,  64)
GRAY      = (128, 128, 128)
LIGHTGRAY = (212, 208, 200)

class PygButton(object):
    def __init__(
        self, rect=None, caption='', bgcolor=None, fgcolor=None, 
        font=None, normal=None, down=None, highlight=None
    ):
        """Create a new button object. Parameters:
            rect - The size and position of the button as a pygame.Rect object
                or 4-tuple of integers.
            caption - The text on the button (default is blank)
            bgcolor - The background color of the button (default is a light
                gray color)
            fgcolor - The foreground color (i.e. the color of the text).
                Default is black.
            font - The pygame.font.Font object for the font of the text.
                Default is freesansbold in point 14.
            normal - A pygame.Surface object for the button's normal
                appearance.
            down - A pygame.Surface object for the button's pushed down
                appearance.
            highlight - A pygame.Surface object for the button's appearance
                when the mouse is over it.
            If the Surface objects are used, then the caption, bgcolor,
            fgcolor, and font parameters are ignored (and vice versa).
            Specifying the Surface objects lets the user use a custom image
            for the button.
            The normal, down, and highlight Surface objects must all be the
            same size as each other. Only the normal Surface object needs to
            be specified. The others, if left out, will default to the normal
            surface.
            """
        if rect:
            self._rect = pygame.Rect(rect)
        else:
            self._rect = pygame.Rect(0, 0, 30, 60)

        self._def_caption = caption
        self._caption = caption
        self._bgcolor = bgcolor or LIGHTGRAY
        self._fgcolor = fgcolor or BLACK
        self._font = font or pygame.font.Font('freesansbold.ttf', 14)

        # tracks the state of the button
        self.buttonDown = False
        # is the button currently pushed down?
        self.mouseOverButton = False
        # is the mouse currently hovering over the button?
        self.lastMouseDownOverButton = False
        # was the last mouse down event over the mouse button? (Used to track clicks.)
        self._visible = True
        # is the button visible
        self.customSurfaces = False
        # button starts as a text button instead of having custom images for each surface

        if normal is None:
            # create the surfaces for a text button
            self.surfaceNormal = pygame.Surface(self._rect.size)
            self.surfaceDown = pygame.Surface(self._rect.size)
            self.surfaceHighlight = pygame.Surface(self._rect.size)
            self._update() # draw the initial button images
        else:
            # create the surfaces for a custom image button
            self.setSurfaces(normal, down, highlight)

    def handleEvent(self, eventObj):
        """All MOUSEMOTION, MOUSEBUTTONUP, MOUSEBUTTONDOWN event objects
        created by Pygame should be passed to this method. handleEvent() will
        detect if the event is relevant to this button and change its state.
        There are two ways that your code can respond to button-events. One is
        to inherit the PygButton class and override the mouse*() methods. The
        other is to have the caller of handleEvent() check the return value
        for the strings 'enter', 'move', 'down', 'up', 'click', or 'exit'.
        Note that mouseEnter() is always called before mouseMove(), and
        mouseMove() is always called before mouseExit(). Also, mouseUp() is
        always called before mouseClick().
        buttonDown is always True when mouseDown() is called, and always False
        when mouseUp() or mouseClick() is called. lastMouseDownOverButton is
        always False when mouseUp() or mouseClick() is called."""

        if eventObj.type not in (MOUSEMOTION, MOUSEBUTTONUP, MOUSEBUTTONDOWN) or not self._visible:
            # The button only cares bout mouse-related events (or no events, if it is invisible)
            return []

        retVal = []

        hasExited = False
        if not self.mouseOverButton and self._rect.collidepoint(eventObj.pos):
            # if mouse has entered the button:
            self.mouseOverButton = True
            self.mouseEnter(eventObj)
            retVal.append('enter')
        elif self.mouseOverButton and not self._rect.collidepoint(eventObj.pos):
            # if mouse has exited the button:
            self.mouseOverButton = False
            hasExited = True 
            # call mouseExit() later, since we want mouseMove() to be handled before mouseExit()

        if self._rect.collidepoint(eventObj.pos):
            # if mouse event happened over the button:
            if eventObj.type == MOUSEMOTION:
                self.mouseMove(eventObj)
                retVal.append('move')
            elif eventObj.type == MOUSEBUTTONDOWN:
                self.buttonDown = True
                self.lastMouseDownOverButton = True
                self.mouseDown(eventObj)
                retVal.append('down')
        else:
            if eventObj.type in (MOUSEBUTTONUP, MOUSEBUTTONDOWN):
                # if an up/down happens off the button, then the next up won't cause mouseClick()
                self.lastMouseDownOverButton = False

        # mouse up is handled whether or not it was over the button
        doMouseClick = False
        if eventObj.type == MOUSEBUTTONUP:
            if self.lastMouseDownOverButton:
                doMouseClick = True
            self.lastMouseDownOverButton = False

            if self.buttonDown:
                self.buttonDown = False
                self.mouseUp(eventObj)
                retVal.append('up')

            if doMouseClick:
                self.buttonDown = False
                self.mouseClick(eventObj)
                retVal.append('click')

        if hasExited:
            self.mouseExit(eventObj)
            retVal.append('exit')

        return retVal

    def draw(self, surfaceObj):
        """Blit the current button's appearance to the surface object."""
        if self._visible:
            if self.buttonDown:
                surfaceObj.blit(self.surfaceDown, self._rect)
            elif self.mouseOverButton:
                surfaceObj.blit(self.surfaceHighlight, self._rect)
            else:
                surfaceObj.blit(self.surfaceNormal, self._rect)


    def _update(self):
        """Redraw the button's Surface object. Call this method when the button has changed appearance."""
        if self.customSurfaces:
            self.surfaceNormal    = pygame.transform.smoothscale(self.origSurfaceNormal, self._rect.size)
            self.surfaceDown      = pygame.transform.smoothscale(self.origSurfaceDown, self._rect.size)
            self.surfaceHighlight = pygame.transform.smoothscale(
                self.origSurfaceHighlight, self._rect.size
            )
            return

        w = self._rect.width # syntactic sugar
        h = self._rect.height # syntactic sugar

        # fill background color for all buttons
        self.surfaceNormal.fill(self.bgcolor)
        self.surfaceDown.fill(self.bgcolor)
        self.surfaceHighlight.fill(self.bgcolor)

        # draw caption text for all buttons
        captionSurf = self._font.render(self._caption, True, self.fgcolor, self.bgcolor)
        captionRect = captionSurf.get_rect()
        captionRect.center = int(w / 2), int(h / 2)
        self.surfaceNormal.blit(captionSurf, captionRect)
        self.surfaceDown.blit(captionSurf, captionRect)

        # draw border for normal button
        pygame.draw.rect(self.surfaceNormal, BLACK, pygame.Rect((0, 0, w, h)), 1) 
        # black border around everything
        pygame.draw.line(self.surfaceNormal, WHITE, (1, 1), (w - 2, 1))
        pygame.draw.line(self.surfaceNormal, WHITE, (1, 1), (1, h - 2))
        pygame.draw.line(self.surfaceNormal, DARKGRAY, (1, h - 1), (w - 1, h - 1))
        pygame.draw.line(self.surfaceNormal, DARKGRAY, (w - 1, 1), (w - 1, h - 1))
        pygame.draw.line(self.surfaceNormal, GRAY, (2, h - 2), (w - 2, h - 2))
        pygame.draw.line(self.surfaceNormal, GRAY, (w - 2, 2), (w - 2, h - 2))

        # draw border for down button
        pygame.draw.rect(self.surfaceDown, BLACK, pygame.Rect((0, 0, w, h)), 1) 
        # black border around everything
        pygame.draw.line(self.surfaceDown, WHITE, (1, 1), (w - 2, 1))
        pygame.draw.line(self.surfaceDown, WHITE, (1, 1), (1, h - 2))
        pygame.draw.line(self.surfaceDown, DARKGRAY, (1, h - 2), (1, 1))
        pygame.draw.line(self.surfaceDown, DARKGRAY, (1, 1), (w - 2, 1))
        pygame.draw.line(self.surfaceDown, GRAY, (2, h - 3), (2, 2))
        pygame.draw.line(self.surfaceDown, GRAY, (2, 2), (w - 3, 2))

        # draw border for highlight button
        self.surfaceHighlight = self.surfaceNormal


    def mouseClick(self, event):
        pass # This class is meant to be overridden.
    def mouseEnter(self, event):
        pass # This class is meant to be overridden.
    def mouseMove(self, event):
        pass # This class is meant to be overridden.
    def mouseExit(self, event):
        pass # This class is meant to be overridden.
    def mouseDown(self, event):
        pass # This class is meant to be overridden.
    def mouseUp(self, event):
        pass # This class is meant to be overridden.


    def setSurfaces(self, normalSurface, downSurface=None, highlightSurface=None):
        """Switch the button to a custom image type of button (rather than a
        text button). You can specify either a pygame.Surface object or a
        string of a filename to load for each of the three button appearance
        states."""
        if downSurface is None:
            downSurface = normalSurface
        if highlightSurface is None:
            highlightSurface = normalSurface

        if type(normalSurface) == str:
            self.origSurfaceNormal = pygame.image.load(normalSurface)
        if type(downSurface) == str:
            self.origSurfaceDown = pygame.image.load(downSurface)
        if type(highlightSurface) == str:
            self.origSurfaceHighlight = pygame.image.load(highlightSurface)

        if self.origSurfaceNormal.get_size() != self.origSurfaceDown.get_size() \
            != self.origSurfaceHighlight.get_size():
            raise Exception('foo')

        self.surfaceNormal = self.origSurfaceNormal
        self.surfaceDown = self.origSurfaceDown
        self.surfaceHighlight = self.origSurfaceHighlight
        self.customSurfaces = True
        self._rect = pygame.Rect(
            (self._rect.left, self._rect.top, 
             self.surfaceNormal.get_width(), self.surfaceNormal.get_height())
        )

    def reset_text(self):
        self.caption = self.def_caption

    @property
    def def_caption(self):
        return self._def_caption

    @def_caption.setter
    def def_caption(self, caption_text):
        self._def_caption = caption_text
        self.caption = caption_text

    @property
    def caption(self):
        return self._caption

    @caption.setter
    def caption(self, caption_text):
        self.customSurfaces = False
        self._caption = caption_text
        self._update()

    @property
    def rect(self):
        return self._rect

    @rect.setter
    def rect(self, new_rect):
        # Note that changing the attributes of the Rect won't update the button. 
        # You have to re-assign the rect member.
        self._update()
        self._rect = new_rect

    @property
    def visible(self):
        return self._visible

    @visible.setter
    def visible(self, setting):
        self._visible = setting

    @property
    def fgcolor(self):
        return self._fgcolor

    @fgcolor.setter
    def fgcolor(self, setting):
        self.customSurfaces = False
        self._fgcolor = setting
        self._update()

    @property
    def bgcolor(self):
        return self._bgcolor

    @bgcolor.setter
    def bgcolor(self, setting):
        self.customSurfaces = False
        self._bgcolor = setting
        self._update()

    @property
    def font(self):
        return self._font

    @font.setter
    def font(self, setting):
        self.customSurfaces = False
        self._font = setting
        self._update()


In [3]:
class Rank(Enum):
    Two = 2, '2'
    Three = 3, '3'
    Four = 4, '4'
    Five = 5, '5'
    Six = 6, '6'
    Seven = 7, '7'
    Eight = 8, '8'
    Nine = 9, '9'
    Ten = 10, '10'
    Jack = 11, 'J'
    Queen = 12, 'Q'
    King = 13, 'K'
    Ace = 14, 'A'

    def __str__(self):
        return self.value[1]

    def __int__(self):
        return self.value[0]


class Suit(Enum):
    Clubs = 'C'
    Diamonds = 'D'
    Hearts = 'H'
    Spades = 'S'

    def __str__(self):
        return self.value


class Deck:
    def __init__ (self):
        self.shuffle()

    def shuffle(self):
        self.deck = [(rank, suit) for rank in Rank for suit in Suit]
        random.shuffle(self.deck)

    def __len__ (self):
        return len(self.deck)

    def __str__(self):
        return str([str(rank) + str(suit) for rank, suit in self.deck])

    def deal (self, *args):
        if len(self) == 0:
            return None
        elif args:
            return (self.deck.pop() for _ in range(args[0]))
        else:
            return (self.deck.pop(), )


def hand_result(sorted_hand):
    rank_list = [card.rank for card in sorted_hand]
    rank_counts = Counter(rank_list)
    suit_check = sorted_hand[0].suit
    for card in sorted_hand:
        if len(sorted_hand) == 3:
            break
        if card.suit != suit_check:
            break
    else:
        if str(sorted_hand[0].rank) == 'A':
            if int(sorted_hand[4]) == 10:
                return 'Royal Flush', 8 * 13 ** 5 + score_hand(sorted_hand)
            elif int(sorted_hand[1]) == 5 and int(sorted_hand[4]) == 2:
                return 'Five High Straight Flush', 8 * 13 ** 5 + score_hand(sorted_hand[1:] + [1])
            else:
                return 'Ace High Flush', 5 * 13 ** 5 + score_hand(sorted_hand)
        elif int(sorted_hand[0]) - int(sorted_hand[4]) == 4:
            return f'{sorted_hand[0].rank.name} High Straight Flush', 8 * 13 ** 5 + score_hand(sorted_hand)
        else:
            return f'{sorted_hand[0].rank.name} High Flush', 5 * 13 ** 5 + score_hand(sorted_hand)
    if len(rank_counts) == 1:
        return (
            f'Three of a Kind, {sorted_hand[0].rank.name}s', 
            3 * 13 ** 5 + score_hand(sorted_hand + [1, 1])
        )
    if len(rank_counts) == 2:
        if 4 in rank_counts.values():
            return f'Four of a Kind, {sorted_hand[0].rank.name}s', 7 * 13 ** 5 + score_hand(sorted_hand)
        elif 3 in rank_counts.values():
            return (
                f'Full House {sorted_hand[0].rank.name}s over {sorted_hand[4].rank.name}s', 
                6 * 13 ** 5 + score_hand(sorted_hand)
            )
        else:
            return f'Pair of {sorted_hand[0].rank.name}s', 1 * 13 ** 5 + score_hand(sorted_hand + [1, 1])
    elif len(rank_counts) == 3:
        if 3 in rank_counts.values():
            return f'Three of a Kind, {sorted_hand[0].rank.name}s', 3 * 13 ** 5 + score_hand(sorted_hand)
        elif 2 in rank_counts.values():
            return (
                f'Two Pair, {sorted_hand[0].rank.name}s over {sorted_hand[2].rank.name}s', 
                2 * 13 ** 5 + score_hand(sorted_hand)
            )
        else:
            return f'{sorted_hand[0].rank.name} High', score_hand(sorted_hand + [1, 1])
    elif len(rank_counts) == 4:
        return  f'Pair of {sorted_hand[0].rank.name}s', 1 * 13 ** 5 + score_hand(sorted_hand)
    else:
        if int(sorted_hand[0]) - int(sorted_hand[4]) == 4:
            return f'{sorted_hand[0].rank.name} High Straight', 4 * 13 ** 5 + score_hand(sorted_hand)
        elif int(sorted_hand[0]) == 14 and int(sorted_hand[1]) == 5 and int(sorted_hand[4]) == 2:
            return 'Five High Straight', 4 * 13 ** 5 + score_hand(sorted_hand[1:] + [1])
        else:
            return f'{sorted_hand[0].rank.name} High', score_hand(sorted_hand)


def score_hand(hand):
    return (int(hand[0]) * 13 ** 4 
           + int(hand[1]) * 13 ** 3 
           + int(hand[2]) * 13 ** 2 
           + int(hand[3]) * 13
           + int(hand[4]))

In [8]:
class Player:
    def __init__(self, name, points=0):
        self.name = name
        self.points = points

    def __str__(self):
        return f"{self.name} has {self.points} points"


class CardSprite:
    def __init__ (self, rank, suit):
        self.rank = rank
        self.suit = suit
        self.draggable = True
        self.dragging = False
        self.image, self._rect = load_png(str(self) + '.png', dimensions=(CARD_WIDTH, CARD_HEIGHT))
        pygame.draw.rect(self.image, BLACK, self.rect, 1)

    def __str__(self):
        return str(self.rank) + str(self.suit)

    def __int__(self):
        return int(self.rank)

    def draw(self, surface):
        surface.blit(self.image, self.rect)
    
    @property
    def rect(self):
        return self._rect

    @rect.setter
    def rect(self, rect):
        self._rect = pygame.Rect(rect)


class CardHolder(pygame.sprite.Sprite):
    def __init__(self, lefttop=None):
        if lefttop:
            self._rect = pygame.Rect(lefttop, (CARD_WIDTH, CARD_HEIGHT))
        else:
            self._rect = pygame.Rect((0, 0), (CARD_WIDTH, CARD_HEIGHT))
        self.image = pygame.Surface(self.rect.size)
        self.image.fill(LIME)
        self._hitbox = self.rect.inflate(SPACE * 5, SPACE * 5)
        self._card = None
        pygame.sprite.Sprite.__init__(self)

    @property
    def rect(self):
        return self._rect

    @rect.setter
    def rect(self, lefttop):
        self._rect = pygame.Rect(lefttop, (CARD_WIDTH, CARD_HEIGHT))
        self._hitbox = self.rect.inflate(SPACE * 5, SPACE * 5)

    @property
    def card(self):
        return self._card

    @card.setter
    def card(self, card):
        if card:
            card.rect = self.rect
        self._card = card

class BoardRow:
    def __init__(self, row_name, slots):
        self.collapsed = False
        self._slots = []
        self.name = row_name
        self._rect = pygame.Rect((0, 0), ((CARD_WIDTH + SPACE) * len(self._slots) - SPACE, CARD_HEIGHT))
        self.slots = int(slots)

    def __len__(self):
        return len(self._slots)

    def __getitem__(self, key):
        return self._slots[key]

    def __str__(self):
        string = ""
        for slot in self:
            if string:
                string += ", "
            if slot.card:
                string += str(slot.card)
            else:
                string += "Empty"
        return string

    def draw(self, surface):
        if self.name != 'hand' and not self.collapsed:
            for slot in self:
                surface.blit(slot.image, slot.rect)
        for card in self.cards:
            card.draw(surface)

    def get_slots(self, slots):
        return [CardHolder() for _ in range(slots)]

    def set_rects(self):
        for count, slot in enumerate(self):
            if not self.collapsed:
                slot.rect = (
                    self._rect.left + (CARD_WIDTH + SPACE) * count, 
                    self._rect.top
                )
                slot.card = slot.card
            else:
                slot.rect = (
                    self._rect.left + (SPACE * count),
                    self._rect.top
                )
                slot.card = slot.card

    def organize(self):
        sorted_hand = sorted(self.cards, key=int, reverse=True)
        rank_list = [card.rank for card in sorted_hand]
        rank_counts = Counter(rank_list)
        kickers = []
        if 3 in rank_counts.values():
            if 2 in rank_counts.values():
                if sorted_hand[0].rank == sorted_hand[2].rank:
                    for slot, card in zip(self.slots, sorted_hand):
                        slot.card = card
                else:
                    sorted_hand = sorted_hand[2:] + sorted_hand[:2]
                    for slot, card in zip(self.slots, sorted_hand):
                        slot.card = card
        for rank, count in rank_counts.items():
            if count == 1:
                for card in sorted_hand:
                    if card.rank == rank:
                        sorted_hand.remove(card) 
                        kickers.append(card)
        kickers = sorted(kickers, key=int, reverse=True)
        sorted_hand += kickers
        for count in range(len(sorted_hand)):
            self[count].card = sorted_hand[count]

    def result(self):
        self.organize()
        self.result, self.score = hand_result(self.cards)

    @property
    def rect(self):
        return self._rect

    @rect.setter
    def rect(self, topcenter):
        top, centerx = topcenter
        if not self.collapsed:
            self._rect = pygame.Rect(
                (0, top), 
                ((CARD_WIDTH + SPACE) * len(self._slots) - SPACE, CARD_HEIGHT)
            )
        else:
            self._rect = pygame.Rect(
                (0, top), 
                (CARD_WIDTH + (SPACE * (len(self._slots) - 1)), CARD_HEIGHT)
            )
        self._rect.centerx = centerx
        self.set_rects()

    @property
    def slots(self):
        return self._slots
    
    @slots.setter
    def slots(self, slots):
        if slots < len(self.cards):
            raise RunTimeError("More cards in row than slots")
        cards = self.cards
        self._slots = self.get_slots(slots)
        self.rect = (self._rect.top, self._rect.centerx)
        for count in range(len(cards)):
            self[count].card = cards[count]

    @property
    def cards(self):
        return [slot.card for slot in self if slot.card]
    
    @cards.setter
    def cards(self, new_cards):
        new_count = len(new_cards)
        for old_card in new_cards:
            if old_card in self.cards:
                new_count -= 1
        if new_count <= len(self) - len(self.cards):
            for new_card in new_cards:
                if new_card not in self.cards:
                    for slot in self:
                        if not slot.card:
                            slot.card = new_card
                            break
        else:
            new_cards += [None] * (len(self) - len(new_cards))
            removed_cards = self.cards
            for count in range(len(new_cards)):
                self[count].card = new_cards[count]
                if new_cards[count] in removed_cards:
                    removed_cards.remove(new_cards[count])
            self.dropped_cards = removed_cards


class Board:
    def __init__(self, player=None, lefttop=None):
        """Create a new board object.
            """
        if lefttop:
            self._rect = pygame.Rect(lefttop, (BOARD_WIDTH, BOARD_HEIGHT))
        else:
            self._rect = pygame.Rect((SCREEN_WIDTH - BOARD_WIDTH)//2, SPACE, BOARD_WIDTH, BOARD_HEIGHT)

        self.player = player or Player('Player')
        self.visible = True

        self._rows = {
            'top':    BoardRow(   'top', 3),
            'middle': BoardRow('middle', 5),
            'bottom': BoardRow('bottom', 5),
            'hand':   BoardRow(  'hand', 5)
        }
        self._update()
        self.deal(5)

    def __len__(self):
        return len(self.rows)

    def __getitem__(self, key):
        if isinstance(key, (int, slice)):
            return [self.rows['top'], self.rows['middle'], self.rows['bottom']][key]
        else:
            return self.rows[key]

    def __iter__(self):
        for row in self.rows.values():
            yield row

    def draw(self, surface, hand=True):
        """Draws slots and cards onto surface"""
        if self.visible:
            if hand:
                for row in self:
                    row.draw(surface)
            else:
                for row in self:
                    if row.name != 'hand':
                        row.draw(surface)
            for row in self:
                for card in row.cards:
                    if card.dragging:
                        card.draw(surface)

    def _update(self):
        """Call this method when you move the board."""
        top = self._rect.top
        self['top'].rect = (top, self._rect.centerx)
        self['middle'].rect = (top + CARD_HEIGHT + SPACE, self._rect.centerx)
        self['bottom'].rect = (top + CARD_HEIGHT * 2 + SPACE * 2, self._rect.centerx)
        if self.hand:
            self.hand.rect = (SCREEN_HEIGHT - CARD_HEIGHT - SPACE, self._rect.centerx)

    def handle_dragging(self, event):
        if event.type == pygame.MOUSEBUTTONDOWN:
            if event.button == 1:
                for row in self:
                    for slot in row:
                        if slot.card and (slot.card.draggable or row.name != 'hand'):
                            if slot.card.rect.collidepoint(event.pos) and not row.collapsed:
                                slot.card.dragging = True
                                mouse_x, mouse_y = event.pos
                                global offset_x, offset_y
                                offset_x = slot.card.rect.x - mouse_x
                                offset_y = slot.card.rect.y - mouse_y
        elif event.type == pygame.MOUSEBUTTONUP:
            if event.button == 1:
                move_card = None
                for old_row in self:
                    for old_slot in old_row:
                        if old_slot.card and old_slot.card.dragging:
                            move_card = old_slot.card
                            move_card.dragging = False
                            break
                    if move_card:
                        break
                if move_card:
                    if old_row.name == 'hand':
                        for new_row in self:
                            for new_slot in new_row:
                                if new_slot._hitbox.contains(move_card.rect):
                                    if not new_slot.card:
                                        new_slot.card = move_card
                                        old_slot.card = None
                                        break
                                    else:
                                        if len(new_row.cards) != len(new_row.slots):
                                            for slot in reversed(new_row):
                                                if not slot.card:
                                                    slot.card = new_slot.card
                                                    new_slot.card = move_card
                                                    old_slot.card = None
                                                    break
                                            if not old_slot.card:
                                                break
                            if not old_slot.card:
                                break
                        else:
                            old_slot.card = move_card
                    else:
                        for new_row in self:
                            for new_slot in new_row:
                                if new_slot._hitbox.contains(move_card.rect):
                                    if new_row is old_row:
                                        old_slot.card = new_slot.card
                                        new_slot.card = move_card
                                        break
                                    elif move_card.draggable:
                                        if len(new_row.cards) != len(new_row):
                                            for slot in reversed(new_row):
                                                if not slot.card:
                                                    slot.card = new_slot.card
                                                    new_slot.card = move_card
                                                    old_slot.card = None
                                                    break
                                            if not old_slot.card:
                                                break
                            else:
                                continue
                            break
                        else:
                            if move_card.draggable:
                                for new_slot in self.hand:
                                    if not new_slot.card:
                                        new_slot.card = move_card
                                        old_slot.card = None
                                        break
                            else:
                                old_slot.card = move_card
        elif event.type == pygame.MOUSEMOTION:
            mouse_x, mouse_y = event.pos
            for row in self:
                for slot in row:
                    if slot.card and slot.card.dragging:
                        if row.name != 'hand':
                            slot.card.rect.x = mouse_x + offset_x
                        if slot.card.draggable:
                            slot.card.rect.x = mouse_x + offset_x
                            slot.card.rect.y = mouse_y + offset_y

    def lock(self):
        for row in self:
            for card in row.cards:
                card.draggable = False

    def empty_hand(self):
        for slot in self.hand:
            if slot.card:
                slot.card = None

    def deal(self, num_cards):
        self.empty_hand()
        self.hand = num_cards
        cards = [CardSprite(rank, suit) for rank, suit in Board.deck.deal(num_cards)]
        for slot, card in zip(self.hand, cards):
            slot.card = card
        self.hand.organize()

    def collapse_full(self):
        for row in self:
            if row.name != 'hand':
                if not row.collapsed and len(row) == len(row.cards):
                    row.organize()
                    row.collapsed = True
                    self._update()

    @property
    def rows(self):
        return self._rows

    @rows.setter
    def rows(self, rows):
        self._rows = rows

    @property
    def rect(self):
        return self._rect

    @rect.setter
    def rect(self, topcenter):
        top, centerx = topcenter
        self._rect.top = top
        self._rect.centerx = centerx
        self._update()
    
    @property
    def hand(self):
        if 'hand' in self.rows.keys():
            return self['hand']
        else:
            return None
    
    @hand.setter
    def hand(self, size):
        if len(self['hand'].slots) != size:
            self['hand'].slots = size
            self._update()

    @hand.deleter
    def hand(self):
        self.rows.pop('hand', None)


def load_png(name, dimensions=None):
    """ Load image and return image object"""
    fullname = os.path.join('images', name)
    try:
        image = pygame.image.load(fullname)
        if image.get_alpha() is None:
            image = image.convert()
        else:
            image = image.convert_alpha()
        if dimensions:
            image = pygame.transform.scale(image, dimensions)
    except pygame.error as message:
        print('Cannot load image:', fullname)
        raise SystemExit(message)
    return image, image.get_rect()


In [9]:
def score_boards(boards):
    for board in boards:
        del board.hand
        print(board.player.name)
        for row in board:
            row.result()
            print(row.name + ':')
            print(row.result)
            print(row)
            print()
        if (
            board['top'].score > board['middle'].score
            or board['top'].score > board['bottom'].score
            or board['middle'].score > board['bottom'].score
        ):
            for row in board:
                row.score = 0
            print("FOUL")
        print()
        print()


def winner(boards):
    if len(boards) > 1:
        for row_count in range(3):
            row_name = boards[0][row_count].name
            row_scores = {}
            for board in boards:
                row_scores[board.player.name] = board[row_count].score
            print('Rankings ' + row_name + ':')
            row_scores = {
                k: v for k, v in sorted(row_scores.items(), key=lambda item: item[1], reverse=True)
            }
            comparators = []
            result = ""
            for count, score in enumerate(list(row_scores.values())[:-1]):
                if score == list(row_scores.values())[count + 1]:
                    comparators.append('= ')
                else:
                    comparators.append('> ')
            for count, name in enumerate(row_scores.keys()):
                result += name + ' '
                if count < len(comparators):
                    result += comparators[count]
            print(result)
            print()
                
    else:
        if boards[0]['top'].score == 0:
            lose_message = [
                "You fucking suck", 
                "Eat a dick you bad", 
                "Why you even try?", 
                "It's okay you'll get em next time",
                "Sorry you fouled out"
            ]
            print(random.choice(lose_message))
        else:
            win_message=[
                "You did okay I guess",
                "Nice",
                "Bravisimo, fantastico, alrightasaur",
                "Get it",
                "I've seen better"
            ]
            print(random.choice(win_message))


In [10]:
def play(screen, players):
    Board.deck = Deck()
    boards = [Board(player) for player in players]

    background = pygame.Surface(screen.get_size())
    background = background.convert()
    background.fill(GREEN)

    next_rect = Rect((SCREEN_WIDTH - BUTTON_WIDTH - SPACE, SCREEN_HEIGHT / 2, BUTTON_WIDTH, BUTTON_HEIGHT))
    next_button = PygButton(next_rect, caption="End Turn")
    next_button.visible = False

    show1_rect = Rect((SCREEN_WIDTH - BUTTON_WIDTH - SPACE, SPACE * 2, BUTTON_WIDTH, BUTTON_HEIGHT))
    show1_button = PygButton(show1_rect)
    show1_button.visible = False

    show2_rect = Rect(
        (SCREEN_WIDTH - BUTTON_WIDTH - SPACE, BUTTON_HEIGHT + SPACE * 3, BUTTON_WIDTH, BUTTON_HEIGHT)
    )
    show2_button = PygButton(show2_rect)
    show2_button.visible = False

    if len(boards) > 1:
        show1_button.visible = True
    if len(boards) > 2:
        show2_button.visible = True

    clock = pygame.time.Clock()

    for turns in range(5):
        for play_board in boards:
            other_boards = [oth_board for oth_board in boards if oth_board is not play_board]
            if show1_button.visible:
                show1_button.def_caption = "Show " + other_boards[0].player.name
            if show2_button.visible:
                show2_button.def_caption = "Show " + other_boards[1].player.name
            show_board = play_board
            running = True
            while running:
                if turns == 0:
                    if len(play_board.hand.cards) == 0:
                         next_button.visible = True
                elif len(play_board.hand.cards) > 1:
                    next_button.visible = False
                    for slot in play_board.hand:
                        if slot.card:
                            slot.card.draggable = True
                else:
                    next_button.visible = True
                    for slot in play_board.hand:
                        if slot.card:
                            slot.card.draggable = False
                for event in pygame.event.get():
                    if event.type == pygame.QUIT:
                        sys.exit()
                    elif 'click' in show1_button.handleEvent(event):
                        if show2_button.caption != show2_button.def_caption:
                            show2_button.reset_text()
                        if show1_button.caption == show1_button.def_caption:
                            show_board = other_boards[0]
                            show1_button.caption = 'Return'
                        else:
                            show_board = play_board
                            show1_button.reset_text()
                    elif 'click' in show2_button.handleEvent(event):
                        if show1_button.caption != show1_button.def_caption:
                            show1_button.reset_text()
                        if show2_button.caption == show2_button.def_caption:
                            show_board = other_boards[1]
                            show2_button.caption = 'Return'
                        else:
                            show_board = play_board
                            show2_button.reset_text()
                    elif 'click' in next_button.handleEvent(event):
                        if turns == 4 and play_board is boards[-1]:
                            next_button.visible = False
                            running = False
                        elif next_button.caption == "End Turn":
                            if len(boards) > 1:
                                play_board.lock()
                                play_board.empty_hand()
                                next_button.caption = "Next Player"
                            else:
                                next_button.visible = False
                                running = False
                        else:
                            next_button.visible = False
                            next_button.caption = "End Turn"
                            running = False
                    elif show_board is play_board:
                        play_board.handle_dragging(event)
                screen.blit(background, (0, 0))
                next_button.draw(screen)
                show1_button.draw(screen)
                show2_button.draw(screen)
                if show_board is play_board:
                    show_board.draw(screen)
                else:
                    show_board.draw(screen, hand=False)
                pygame.display.update()
                clock.tick(FPS)
            play_board.collapse_full()
            play_board.lock()
            if turns != 4:
                play_board.deal(3)
    return win_screen(screen, boards)


def win_screen(screen, boards):
    score_boards(boards)
    winner(boards)

    background = pygame.Surface(screen.get_size())
    background = background.convert()
    background.fill(GREEN)

    replay_rect = Rect(
        (
            SCREEN_WIDTH - BUTTON_WIDTH - SPACE, 
            SPACE, 
            BUTTON_WIDTH, 
            BUTTON_HEIGHT
        )
    )
    replay_button = PygButton(replay_rect, caption="Play Again")

    home_rect = Rect(
        (
            SCREEN_WIDTH - BUTTON_WIDTH - SPACE, 
            BUTTON_HEIGHT + SPACE * 2, 
            BUTTON_WIDTH, 
            BUTTON_HEIGHT
        )
    )
    home_button = PygButton(home_rect, caption="Home")

    for count, board in enumerate(boards):
        board.rect = (SCREEN_HEIGHT / 5, (SCREEN_WIDTH * (count + 1))/(len(boards) + 1))

    player_text = [FONT.render(board.player.name, fgcolor=LIME) for board in boards]
    i = 0
    for image, rect in player_text:
        rect.top = SPACE * 2
        rect.centerx = (SCREEN_WIDTH * (i + 1)) / (len(boards) + 1)
        i += 1

    fouls = []
    for board in boards:
        if board['top'].score == 0:
            fouls.append(FONT.render('FOUL', fgcolor=RED))
        else:
            fouls.append(None)
    i = 0
    for foul in fouls:
        if foul:
            image, rect = foul
            rect.top = 32 + SPACE * 3
            rect.centerx = (SCREEN_WIDTH * (i + 1)) / (len(boards) + 1)
        i += 1

    clock = pygame.time.Clock()

    while True:
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                sys.exit()
            elif 'click' in replay_button.handleEvent(event):
                return (board.player for board in boards)
            elif 'click' in home_button.handleEvent(event):
                return
        screen.blit(background, (0, 0))
        for board in boards:
            board.draw(screen)
        for image, rect in player_text:
            screen.blit(image, rect)
        for foul in fouls:
            if foul:
                image, rect = foul
                screen.blit(image, rect)
        replay_button.draw(screen)
        home_button.draw(screen)
        pygame.display.update()
        clock.tick(FPS)


def home_screen(screen):
    background = pygame.Surface(screen.get_size())
    background = background.convert()
    background.fill(GREEN)

    titlefont = pygame.font.Font(None, 124)
    title = titlefont.render("Pineapple", 1, LIME)
    titlepos = title.get_rect()
    titlepos.top = SPACE * 3
    titlepos.centerx = background.get_rect().centerx
    
    play_1_rect = Rect((0, 0, BUTTON_WIDTH, BUTTON_HEIGHT))
    play_1_rect.centerx = background.get_rect().centerx
    play_1_rect.centery = background.get_rect().centery
    button1 = PygButton(play_1_rect, caption='One Player')
    
    play_2_rect = Rect((0, 0, BUTTON_WIDTH, BUTTON_HEIGHT))
    play_2_rect.centerx = background.get_rect().centerx
    play_2_rect.centery = background.get_rect().centery + BUTTON_HEIGHT + SPACE
    button2 = PygButton(play_2_rect, caption='Two Players')
    
    play_3_rect = Rect((0, 0, BUTTON_WIDTH, BUTTON_HEIGHT))
    play_3_rect.centerx = background.get_rect().centerx
    play_3_rect.centery = background.get_rect().centery + (BUTTON_HEIGHT + SPACE) * 2
    button3 = PygButton(play_3_rect, caption='Three Players')

    clock = pygame.time.Clock()

    players = None
    while True:
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                sys.exit()
            elif 'click' in button1.handleEvent(event):
                players = (Player('Player1'),)
            elif 'click' in button2.handleEvent(event):
                players = (Player('Player1'), Player('Player2'))
            elif 'click' in button3.handleEvent(event):
                players = (Player('Player1'), Player('Player2'), Player('Player3'))
        if players: 
            players = play(screen, players)
        else:
            screen.blit(background, (0, 0))
            screen.blit(title, titlepos)
            button1.draw(screen)
            button2.draw(screen)
            button3.draw(screen)
            pygame.display.update()
            clock.tick(FPS)

In [12]:
def main():
    pygame.init()
    logo = pygame.image.load(os.path.join('images', 'PineLogo.png'))
    pygame.display.set_icon(logo)
    pygame.display.set_caption("Pineapple")

    screen = pygame.display.set_mode((SCREEN_WIDTH, SCREEN_HEIGHT))
    try:
        home_screen(screen)
    except SystemExit:
        pygame.display.quit()

if __name__=="__main__":
    main()


Player1
top:
Ace High
AC, 8C, 5D

middle:
Three of a Kind, Queens
QH, QC, QS, 10C, 5C

bottom:
Ace High Flush
AS, KS, JS, 9S, 5S



Nice
Player1
top:
King High
KD, JH, 10D

middle:
Pair of Aces
AD, AC, 6C, 5D, 4H

bottom:
Ten High Straight
10C, 9S, 8S, 7H, 6H



Nice


In [ ]:
class prob:
    def __init__(self, cards, three=False):
        self.cards = cards
        self.three = three

    def possible(self):
        possible = []
        self.organize()
        sorted_hand = self.cards
        rank_list = [card.rank for card in self.cards]
        rank_counts = Counter(rank_list)
        suit_check = sorted_hand[0].suit
        for card in sorted_hand:
            if self.three:
                break
            if card.suit != suit_check:
                break
        else:
            if str(sorted_hand[0].rank) == 'A':
                if int(sorted_hand[-1]) > 9:
                    possible.append('Royal Flush')
                elif int(sorted_hand[1]) < 6:
                    possible.append('Straight Flush')
            elif int(sorted_hand[0]) - int(sorted_hand[-1]) < 5:
                possible.append('Straight Flush')
            else:
                possible.append('High Flush')
        if len(rank_counts) == len(sorted_hand) and not self.three:
            if int(sorted_hand[0]) - int(sorted_hand[-1]) < 5:
                possible.append('Straight')
        if len(rank_counts) < 5:
            possible.append('Pair')
            
            
            
            
            
            
        elif len(rank_counts) == 3:
            possible.append('Two Pair')
            if 4 in rank_counts.values():
                return f'Four of a Kind, {sorted_hand[0].rank.name}s', 7 * 13 ** 5 + score_hand(sorted_hand)
            elif 3 in rank_counts.values():
                return (
                    f'Full House {sorted_hand[0].rank.name}s over {sorted_hand[4].rank.name}s', 
                    6 * 13 ** 5 + score_hand(sorted_hand)
                )
            else:
                return f'Pair of {sorted_hand[0].rank.name}s', 1 * 13 ** 5 + score_hand(sorted_hand + [1, 1])
        elif len(rank_counts) == 3:
            if 3 in rank_counts.values():
                return f'Three of a Kind, {sorted_hand[0].rank.name}s', 3 * 13 ** 5 + score_hand(sorted_hand)
            elif 2 in rank_counts.values():
                return (
                    f'Two Pair, {sorted_hand[0].rank.name}s over {sorted_hand[2].rank.name}s', 
                    2 * 13 ** 5 + score_hand(sorted_hand)
                )
            else:
                return f'{sorted_hand[0].rank.name} High', score_hand(sorted_hand + [1, 1])
        elif len(rank_counts) == 4:
            return  f'Pair of {sorted_hand[0].rank.name}s', 1 * 13 ** 5 + score_hand(sorted_hand)
        else:
            if int(sorted_hand[0]) - int(sorted_hand[4]) == 4:
                return f'{sorted_hand[0].rank.name} High Straight', 4 * 13 ** 5 + score_hand(sorted_hand)
            elif int(sorted_hand[0]) == 14 and int(sorted_hand[1]) == 5 and int(sorted_hand[4]) == 2:
                return 'Five High Straight', 4 * 13 ** 5 + score_hand(sorted_hand[1:] + [1])
            else:
                return f'{sorted_hand[0].rank.name} High', score_hand(sorted_hand)


    def organize(self):
        sorted_hand = sorted(self.cards, key=int, reverse=True)
        rank_list = [card.rank for card in sorted_hand]
        rank_counts = Counter(rank_list)
        kickers = []
        if 3 in rank_counts.values():
            if 2 in rank_counts.values():
                if sorted_hand[0].rank == sorted_hand[2].rank:
                    for slot, card in zip(self.slots, sorted_hand):
                        slot.card = card
                else:
                    sorted_hand = sorted_hand[2:] + sorted_hand[:2]
                    for slot, card in zip(self.slots, sorted_hand):
                        slot.card = card
        for rank, count in rank_counts.items():
            if count == 1:
                for card in sorted_hand:
                    if card.rank == rank:
                        sorted_hand.remove(card) 
                        kickers.append(card)
        kickers = sorted(kickers, key=int, reverse=True)
        sorted_hand += kickers
        self.cards = sorted_hand